Вторая лаба

In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
import pickle

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

In [3]:
df = pd.read_pickle('../data/clean_data.pkl').sample(frac=1, random_state = 2)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 302 entries, 99 to 169
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   age       302 non-null    int8    
 1   sex       302 non-null    category
 2   cp        302 non-null    category
 3   trestbps  302 non-null    int16   
 4   chol      302 non-null    int16   
 5   fbs       302 non-null    category
 6   restecg   302 non-null    category
 7   thalach   302 non-null    int16   
 8   exang     302 non-null    category
 9   oldpeak   302 non-null    float16 
 10  slope     302 non-null    category
 11  ca        302 non-null    category
 12  thal      302 non-null    category
 13  target    302 non-null    category
dtypes: category(9), float16(1), int16(3), int8(1)
memory usage: 7.9 KB


In [5]:
cat_features = df.select_dtypes(include=['category']).columns.to_list()
num_features = df.select_dtypes(include=['number']).columns.to_list()
cat_features

['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'target']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.25, random_state=2)

In [7]:
cat_features = X_train.select_dtypes(include=['category','object']).columns.to_list()
cat_features

['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

In [8]:
num_features = X_train.select_dtypes(include=['number']).columns.to_list()
num_features

['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

На шаге transform для числовых признаков использовать StandardScaler, для категориальных - TargetEncoder(для задачи классификации)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import TargetEncoder

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

In [10]:
s_scaler = StandardScaler()
l_encoder = TargetEncoder() 
classifier= RandomForestClassifier()

In [15]:
# Для удобной работы со столбцами
preprocessor = ColumnTransformer(
    transformers=[
        ('num', s_scaler, num_features),  # преобразования для числовых признаков
        ('cat', l_encoder, cat_features), # преобразования для категориальных признаков
    ],
    remainder='drop' ) # Удаляем столбцы, которые не затронуты преобразования

In [23]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                           ('model', classifier)])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['age', 'trestbps', 'chol',
                                                   'thalach', 'oldpeak']),
                                                 ('cat', TargetEncoder(),
                                                  ['sex', 'cp', 'fbs',
                                                   'restecg', 'exang', 'slope',
                                                   'ca', 'thal'])])),
                ('model', RandomForestClassifier())])

Метрики качетсва для задачи классификации: precision, recall, f1, roc_auc

Precision = tp / (tp + fp), [наихудшее - 0, наилучшее - 1]

Recall = tp / (tp + fn), [наихудшее - 0, наилучшее - 1]

F1 = 2*tp / (2*tp + fp + fn)

Roc_auc = площадь под кривой ROC 

In [24]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

predictions = pipeline.predict(X_test) 

metrics = {}
metrics["precision"] = precision_score(y_test, predictions)   
metrics["recall"] = recall_score(y_test, predictions)
metrics["f1"] = f1_score(y_test, predictions)
metrics["roc_auc"] = roc_auc_score(y_test, predictions)

metrics

{'precision': 0.851063829787234,
 'recall': 0.9302325581395349,
 'f1': 0.8888888888888888,
 'roc_auc': 0.8590556730091613}

In [25]:
confusion_matrix(y_test, predictions)

array([[26,  7],
       [ 3, 40]], dtype=int64)

In [33]:
import mlflow

In [34]:
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

registry_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"
tracking_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"

mlflow.set_tracking_uri(tracking_uri)   
mlflow.set_registry_uri(registry_uri)  

In [29]:
EXPERIMENT_NAME = "heart_project"
RUN_NAME = "baseline model"
REGISTRY_MODEL_NAME = "heart_model_rf"

In [27]:
from mlflow.models import infer_signature
input_example = X_train.head(5)
signature =  infer_signature(model_input = X_train.head(5))

c:\Users\79172\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [30]:
req_file = 'requirements.txt'
art = 'comment.txt'

In [31]:
params_dict = pipeline.get_params()

логирование выдавало ошибку, т.к. не находил файл 'requirements.txt', поэтому я перенесла его в директорию 'research', файл 'comment.txt'создала в этой же папке 'research'

In [57]:
# Когда создаем новый эксперимент, то: 
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(pipeline, 
                             artifact_path="models",
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=req_file
                             )
    mlflow.log_metrics(metrics)
    mlflow.log_artifact(art)
    mlflow.log_params(params_dict)

run = mlflow.get_run(run_id) 
assert (run.info.status =='FINISHED')

2024/11/15 12:14:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run baseline model at: http://127.0.0.1:5000/#/experiments/3/runs/c967d5a4424e41c4b8b304691e68fd36.
2024/11/15 12:14:51 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3.


# Удаление неудач 

Удалить можно или эксперимент целиком, или один run из него, для этого нужно определить их ID:

1) Эту информацию можно получить в Overview (обзор) нажав ЛКП по названию Run. Из таблицы нужны следующие данные:

- Experiment ID = 2

- Run ID = e3faad28c00a4e1cac5a10af2f15e070 

Затем используя 

`mlflow.delete_experiment(2)`

или

`mlflow.delete_run('e3faad28c00a4e1cac5a10af2f15e070 ')`

удаляете необходимое вам.

2) ID эксперимета и run можно узнать с помощью команд:

- `experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id`

- `mlflow.search_runs(experiment_names=[EXPERIMENT_NAME])`

во втором случае результатом выполнения команды будет таблица с данными всего эксперимента, где будет написаны усе ID всех run. Дальнейшее удаление выполняется командами из пункта 1.

# Автологирование

# Модель №2

# Feature engineering

Произведем различные трансформации наших данных, дл генерации новых признаков. 

In [61]:
from sklearn.preprocessing import QuantileTransformer, SplineTransformer, PolynomialFeatures, MinMaxScaler

In [95]:
# создадим копию исходной обучающей выборки
X_train_sklearn = X_train.copy()

In [96]:
# используем PolynomialFeatures для 2-3 числовых признаков.
pf = PolynomialFeatures(degree=2)

In [97]:
X_train_sklearn

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
190,51,0,0,130,305,0,1,142,1,1.200195,1,0,3
103,42,1,2,120,240,1,1,194,0,0.799805,0,0,3
228,59,1,3,170,288,0,0,159,0,0.199951,1,0,3
216,62,0,2,130,263,0,1,97,0,1.200195,1,1,3
224,54,1,0,110,239,0,1,126,1,2.800781,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,57,0,0,120,354,0,1,163,1,0.600098,2,0,2
210,57,1,2,128,229,0,0,150,0,0.399902,1,1,3
204,62,0,0,160,164,0,0,145,0,6.199219,0,3,3
134,41,0,1,126,306,0,1,163,0,0.000000,2,0,2


In [98]:
pf.fit_transform(X_train_sklearn[['age','thalach']])

array([[1.0000e+00, 5.1000e+01, 1.4200e+02, 2.6010e+03, 7.2420e+03,
        2.0164e+04],
       [1.0000e+00, 4.2000e+01, 1.9400e+02, 1.7640e+03, 8.1480e+03,
        3.7636e+04],
       [1.0000e+00, 5.9000e+01, 1.5900e+02, 3.4810e+03, 9.3810e+03,
        2.5281e+04],
       ...,
       [1.0000e+00, 6.2000e+01, 1.4500e+02, 3.8440e+03, 8.9900e+03,
        2.1025e+04],
       [1.0000e+00, 4.1000e+01, 1.6300e+02, 1.6810e+03, 6.6830e+03,
        2.6569e+04],
       [1.0000e+00, 4.4000e+01, 1.7900e+02, 1.9360e+03, 7.8760e+03,
        3.2041e+04]])